# Initialize

In [16]:
from xml.etree import ElementTree, cElementTree
from xml.dom import minidom

root = ElementTree.Element('root')
#print(ElementTree.tostring(root))

# Create XML content

In [17]:
child1 = ElementTree.SubElement(root, 'doc')
child1_1 = ElementTree.SubElement(child1, 'DocChild_1')
child1_1.text = "DocChild_1 content"

# Create tree to XML file

In [18]:
tree = cElementTree.ElementTree(root) # wrap it in an ElementTree instance, and save as XML

# Since ElementTree write() has no pretty printing support, used minidom to beautify the xml
t = minidom.parseString(ElementTree.tostring(root)).toprettyxml()
tree1 = ElementTree.ElementTree(ElementTree.fromstring(t))

tree1.write("filename.xml",encoding='utf-8', xml_declaration=True)